In [2]:
import json
import random
import os
import re
from pathlib import Path
from typing import List, Dict, Any, Tuple
import shutil
import time

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================

# Diretório de onde o script irá ler os JSONs (Treino Geral - AGORA LIMPOS)
INPUT_DIR = Path('data/json/test_ready')

# Diretório para onde os 3 exemplos selecionados serão movidos (Controle)
SAMPLES_DIR = Path('data/json/fewshot_samples')

# Critérios de Seleção
MIN_FATORES_POR_RELATORIO = 2 # Mínimo de fatores contribuintes que cada exemplo deve ter
NUM_EXEMPLOS_DESEJADOS = 3    # Número total de exemplos Few-Shot

# ==============================================================================
# FUNÇÕES DE PROCESSAMENTO (Sem a limpeza individual)
# ==============================================================================

# Nota: A função 'limpar_fator_contribuinte' foi movida para limpador_labels.py
# e será executada separadamente antes deste script.

def carregar_e_filtrar_candidatos(input_dir: Path) -> List[Dict[str, Any]]:
    """
    Carrega todos os JSONs e filtra os que têm o número mínimo de fatores.
    Assume que os arquivos já foram limpos pelo limpador_labels.py.
    """
    candidatos_validos = []
    arquivos_json = list(input_dir.glob("*.json"))

    print(f"📁 Total de arquivos JSON encontrados: {len(arquivos_json)}")

    for arquivo_path in arquivos_json:
        try:
            with open(arquivo_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Acessa a lista de fatores LIMPOS
            fatores_limpos = data.get('conteudo', {}).get('fatores_contribuintes', [])

            # 1. Verifica o critério de tamanho
            if len(fatores_limpos) >= MIN_FATORES_POR_RELATORIO:
                candidatos_validos.append({
                    'path': arquivo_path,
                    'data': data,
                    'fatores': fatores_limpos # Usando a lista LIMPA (Conceito (Classificação))
                })

        except Exception as e:
            print(f"  ❌ Erro ao ler ou analisar {arquivo_path.name}: {e}")

    print(f"🎯 Total de relatórios candidatos (com >= {MIN_FATORES_POR_RELATORIO} fatores limpos): {len(candidatos_validos)}")
    return candidatos_validos

def selecionar_amostra_diversa(candidatos: List[Dict[str, Any]], num_exemplos: int) -> List[Dict[str, Any]]:
    """
    Seleciona 'num_exemplos' relatórios priorizando o conjunto máximo de fatores únicos.
    A classificação (contribuiu) é removida temporariamente para calcular a diversidade por conceito.
    """
    if len(candidatos) < num_exemplos:
        print(f"🛑 Aviso: Apenas {len(candidatos)} candidatos disponíveis. Selecionando todos.")
        return candidatos

    # 1. Contar a frequência de todos os fatores (somente o CONCEITO)
    frequencia_fatores = {}

    def get_conceito_limpo(fator_completo):
        """Extrai apenas o conceito (exclui a classificação entre parênteses)"""
        return fator_completo.split('(')[0].strip().lower()

    for cand in candidatos:
        for fator in cand['fatores']:
            fator_limpo_conceito = get_conceito_limpo(fator)
            frequencia_fatores[fator_limpo_conceito] = frequencia_fatores.get(fator_limpo_conceito, 0) + 1

    # 2. Atribuir um "valor de diversidade" a cada relatório
    for cand in candidatos:
        valor_diversidade = 0
        fatores_unicos_conceito = set()

        for fator in cand['fatores']:
            fator_limpo_conceito = get_conceito_limpo(fator)

            # Fatores menos frequentes são mais valiosos (maior raridade)
            fator_raridade = 1 / (frequencia_fatores.get(fator_limpo_conceito, 1) + 1)
            valor_diversidade += fator_raridade
            fatores_unicos_conceito.add(fator_limpo_conceito)

        cand['score_diversidade'] = valor_diversidade
        cand['fatores_unicos_conceito'] = fatores_unicos_conceito

    # 3. Ordenar e selecionar os 3 mais diversos

    # Ordena todos os candidatos pelo maior score de diversidade
    candidatos_ordenados = sorted(candidatos, key=lambda x: x['score_diversidade'], reverse=True)

    exemplos_selecionados: List[Dict[str, Any]] = []

    # Abordagem gananciosa: pega os K mais diversos
    for i in range(min(num_exemplos, len(candidatos_ordenados))):
        exemplos_selecionados.append(candidatos_ordenados[i])

    return exemplos_selecionados

def contar_labels_e_mover(candidatos: List[Dict[str, Any]], amostra_final: List[Dict[str, Any]]):
    """
    Executa os passos de contar labels e mover a amostra.
    """
    # 1. Contagem Total de Labels (LIMPOS)
    total_labels = sum(len(c.get('fatores', [])) for c in candidatos)
    print(f"\n📊 Contagem total de labels (fatores contribuintes LIMPOS) em {INPUT_DIR}: {total_labels}")

    # 2. Detalhes da Amostra Final
    fatores_unicos_na_amostra = set()
    nomes_arquivos_amostra = []

    for ex in amostra_final:
        fatores_unicos_na_amostra.update(ex['fatores_unicos_conceito'])
        nomes_arquivos_amostra.append(ex['path'].name)

    print(f"\n✨ Amostra selecionada ({len(amostra_final)} relatórios):")
    print(f"   Total de labels únicos (conceitos) na amostra: {len(fatores_unicos_na_amostra)}")
    print(f"   Labels únicos (conceitos, exemplos): {list(fatores_unicos_na_amostra)[:5]}...")

    # 3. Mover os arquivos
    SAMPLES_DIR.mkdir(parents=True, exist_ok=True)
    movidos = 0

    print(f"\n📦 Movendo {len(amostra_final)} arquivos para {SAMPLES_DIR.name}...")
    for ex in amostra_final:
        src_path = ex['path']
        dest_path = SAMPLES_DIR / src_path.name
        try:
            # Move o arquivo (agora ele já está limpo no disco)
            shutil.move(str(src_path), str(dest_path))
            movidos += 1
        except Exception as e:
            print(f"  ❌ Erro ao mover {src_path.name}: {e}")

    print(f"\n✅ Concluído. {movidos} relatórios movidos para a pasta de controle.")
    print("------------------------------------------------------------------")

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================

def main_fewshot_sampler():
    print("ATENÇÃO: Este script assume que 'limpador_labels.py' já foi executado.")
    print("🚀 INICIANDO SELEÇÃO DA AMOSTRA DIVERSA...")

    # 1. Carregar e filtrar documentos
    candidatos = carregar_e_filtrar_candidatos(INPUT_DIR)

    if len(candidatos) < NUM_EXEMPLOS_DESEJADOS:
        print("Não há relatórios candidatos suficientes para a seleção Few-Shot.")
        return

    # 2. Selecionar a amostra diversa
    amostra_final = selecionar_amostra_diversa(candidatos, NUM_EXEMPLOS_DESEJADOS)

    # 3. Contar labels e mover a amostra selecionada
    contar_labels_e_mover(candidatos, amostra_final)

if __name__ == "__main__":
    main_fewshot_sampler()

ATENÇÃO: Este script assume que 'limpador_labels.py' já foi executado.
🚀 INICIANDO SELEÇÃO DA AMOSTRA DIVERSA...
📁 Total de arquivos JSON encontrados: 993
🎯 Total de relatórios candidatos (com >= 2 fatores limpos): 441

📊 Contagem total de labels (fatores contribuintes LIMPOS) em data\json\training_ready: 1410

✨ Amostra selecionada (3 relatórios):
   Total de labels únicos (conceitos) na amostra: 19
   Labels únicos (conceitos, exemplos): ['cultura do grupo de trabalho', 'sistema de apoio', 'processo organizacional', 'pessoal de apoio', 'comunicação']...

📦 Movendo 3 arquivos para fewshot_samples...

✅ Concluído. 3 relatórios movidos para a pasta de controle.
------------------------------------------------------------------
